In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import confusion_matrix

/Users/midhunpk/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# from tensorflow.examples.tutorials.mnist import input_data
# from tensorflow.examples.tutorials.mnist import mnist

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets("MNIST_DATA/", one_hot=False)

Extracting MNIST_DATA/train-images-idx3-ubyte.gz
Extracting MNIST_DATA/train-labels-idx1-ubyte.gz
Extracting MNIST_DATA/t10k-images-idx3-ubyte.gz
Extracting MNIST_DATA/t10k-labels-idx1-ubyte.gz


In [5]:
mnist[0].images.shape

(55000, 784)

In [ ]:
28*28

In [ ]:
plt.imshow(mnist[0].images[0].reshape(28,28))

In [60]:
width=height=28

training_set = mnist[0].images[:1000,].reshape(-1, height, width, 1)
training_labels = mnist[0].labels[:1000]

validation_set = mnist[1].images[1].reshape(-1, height, width, 1)
validation_label = mnist[1].labels

In [61]:
x_train = training_set[:800, :, :]
x_test = training_set[800:, :, : ]
yy_train = training_labels[:800,]
yy_test = training_labels[800:,]




In [120]:
x_train[0]

array([[[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        

In [62]:
y_train = pd.get_dummies(yy_train)
y_test = pd.get_dummies(yy_test)

In [63]:
def create_placeholder(n_H0, n_W0, n_C0, n_y):
  X = tf.placeholder(tf.float32,[None,n_H0, n_W0, n_C0])
  Y = tf.placeholder(tf.float32,[None,n_y])
  return X,Y

In [64]:
def conv_layer(input, size_in, size_out, name='conv'):
  with tf.name_scope(name):
    w = tf.Variable(tf.truncated_normal([28, 28, size_in, size_out], stddev = 0.01), name = 'W')
    b = tf.Variable(tf.constant(0.01,shape=[size_out]),name = 'B')
    conv = tf.nn.conv2d(input, w, strides=[1,1,1,1], padding ="SAME" )
    with tf.name_scope('relu'):
      act = tf.nn.relu(conv + b)
      tf.summary.histogram('weights',w)
      tf.summary.histogram('biases',b)
      tf.summary.histogram('activations',act)
    with tf.name_scope('max_pool'):
        max_pool = tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
    return max_pool

In [65]:
def compute_cost(Z,Y):
  with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z, labels = Y))
    return cost

In [165]:
def iterate_minibatches(inputs,targets,batchsize):#using python operator
  print('iterate')
  print(inputs.shape, targets.shape)
  assert inputs.shape[0] == targets.shape[0]
  m = inputs.shape[0]
  indices = np.arange(m)
  print(indices)
  np.random.shuffle(indices)
  for index in range(batchsize, m - batchsize + 1):
    batch = indices[index:index+batchsize]
    print(batch)
    yield inputs[batch], targets[batch]
    
  if m % batchsize != 0:
    batch = indices[math.floor(m/batchsize)*batchsize:m]
    print(batch)
    yield inputs[batch], targets[batch]

In [159]:
# (m, n_H0, n_W0, n_C0) = x_train.shape
# n_y = y_train.shape[1]

# X,Y= create_placeholder(n_H0, n_W0, n_C0, n_y)

In [160]:
# n_C0

In [161]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((800, 28, 28, 1), (800, 10), (200, 28, 28, 1), (200, 10))

In [162]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [170]:
def model(X_train, X_test, Y_train, Y_test, learning_rate = 0.05, num_epoches = 3, minibatch_size = 50):
  
  global_step = tf.Variable(0, name = 'global_step', trainable = False)
  
  (m, n_H0, n_W0, n_C0) = X_train.shape
  n_y = Y_train.shape[1]
  
  #placeholder
  X,Y = create_placeholder(n_H0, n_W0, n_C0, n_y)
  
  
  #convolution layers
  conv1 = conv_layer(X,     1, 10, 'conv1')
  conv2 = conv_layer(conv1, 10, 16, 'conv2')
  conv3 = conv_layer(conv2, 16, 32, 'conv3')
  conv4 = conv_layer(conv3, 32, 64, 'conv4')
  conv5 = conv_layer(conv4, 64, 128, 'conv5')
  flattend = tf.contrib.layers.flatten(conv5)
  
  

  #fc layers
  fc1 = tf.contrib.layers.fully_connected(flattend, 128, activation_fn=None)
  Z = tf.contrib.layers.fully_connected(fc1, n_y, activation_fn=None)
  prob = tf.nn.softmax(Z, name = 'prob')
  prediction = tf.argmax(Z, 1, name = 'prediction')
  
  
  
  cost = compute_cost(Z,Y)
  
  
  
  with tf.name_scope("optimize"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost, global_step = global_step)
    
    
  with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(Z,1), tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    
    
    
  tf.add_to_collection('X', X)
  tf.add_to_collection('Z', Z)
  tf.add_to_collection('correct_prediction', correct_prediction)
  
  
  
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    batchsize = 10
    print('for loop')
    for epoch in range(num_epoches):
      avg_cost = 0
      print('batch loop')
      for minibatch in iterate_minibatches(X_train, Y_train, batchsize):
        print('inside batch')
        (minibatch_X, minibatch_Y) = minibatch
         
#         print(minibatch_X.shape, minibatch_Y.shape)
        sess.run(optimizer, feed_dict = {X: minibatch_X, Y: minibatch_Y})
        print('optimizer')
        
        avg_cost += sess.run(cost, feed_dict = {X:minibatch_X, Y: minibatch_Y})
        
        iteration = sess.run(global_step)
        print('######', iteration)
        print('inside loop')
        
        

        
    print('Training completed ! !')
    
    #model accuracy

    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))*100
    train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
    test_accuracy = accuracy.eval({X:X_test, Y: Y_test})
    print("Train Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
#     print("Total iterations completed : ", iteration)
    
    
    
    
    #One step forwardprop
    predict_op = tf.argmax(Z, 1) 
    pred = sess.run(predict_op, feed_dict={X: x_test})
    
  return pred
        
        

In [171]:
pred= model(x_train, x_test, y_train, y_test, num_epoches = 3, minibatch_size = 20, learning_rate = 0.005)

for loop
batch loop
iterate
(800, 28, 28, 1) (800, 10)
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 

optimizer
###### 57
inside loop
[ 68 753 314  46 135 776 365 784 504 330]
inside batch
optimizer
###### 58
inside loop
[753 314  46 135 776 365 784 504 330   6]
inside batch
optimizer
###### 59
inside loop
[314  46 135 776 365 784 504 330   6   2]
inside batch
optimizer
###### 60
inside loop
[ 46 135 776 365 784 504 330   6   2  82]
inside batch
optimizer
###### 61
inside loop
[135 776 365 784 504 330   6   2  82 635]
inside batch
optimizer
###### 62
inside loop
[776 365 784 504 330   6   2  82 635 348]
inside batch
optimizer
###### 63
inside loop
[365 784 504 330   6   2  82 635 348 671]
inside batch
optimizer
###### 64
inside loop
[784 504 330   6   2  82 635 348 671 352]
inside batch
optimizer
###### 65
inside loop
[504 330   6   2  82 635 348 671 352 568]
inside batch
optimizer
###### 66
inside loop
[330   6   2  82 635 348 671 352 568 792]
inside batch
optimizer
###### 67
inside loop
[  6   2  82 635 348 671 352 568 792  81]
inside batch
optimizer
###### 68
inside loop
[  2  82 63

optimizer
###### 151
inside loop
[205 637 279 738 571 337 695 376 451  10]
inside batch
optimizer
###### 152
inside loop
[637 279 738 571 337 695 376 451  10 686]
inside batch
optimizer
###### 153
inside loop
[279 738 571 337 695 376 451  10 686 790]
inside batch
optimizer
###### 154
inside loop
[738 571 337 695 376 451  10 686 790 399]
inside batch
optimizer
###### 155
inside loop
[571 337 695 376 451  10 686 790 399 442]
inside batch
optimizer
###### 156
inside loop
[337 695 376 451  10 686 790 399 442 590]
inside batch
optimizer
###### 157
inside loop
[695 376 451  10 686 790 399 442 590 666]
inside batch
optimizer
###### 158
inside loop
[376 451  10 686 790 399 442 590 666 768]
inside batch
optimizer
###### 159
inside loop
[451  10 686 790 399 442 590 666 768 353]
inside batch
optimizer
###### 160
inside loop
[ 10 686 790 399 442 590 666 768 353 398]
inside batch
optimizer
###### 161
inside loop
[686 790 399 442 590 666 768 353 398 684]
inside batch
optimizer
###### 162
inside loop

optimizer
###### 245
inside loop
[663 731 779  33 795 150 418 249 198 437]
inside batch
optimizer
###### 246
inside loop
[731 779  33 795 150 418 249 198 437 678]
inside batch
optimizer
###### 247
inside loop
[779  33 795 150 418 249 198 437 678  70]
inside batch
optimizer
###### 248
inside loop
[ 33 795 150 418 249 198 437 678  70 221]
inside batch
optimizer
###### 249
inside loop
[795 150 418 249 198 437 678  70 221 201]
inside batch
optimizer
###### 250
inside loop
[150 418 249 198 437 678  70 221 201 397]
inside batch
optimizer
###### 251
inside loop
[418 249 198 437 678  70 221 201 397 208]
inside batch
optimizer
###### 252
inside loop
[249 198 437 678  70 221 201 397 208 178]
inside batch
optimizer
###### 253
inside loop
[198 437 678  70 221 201 397 208 178 531]
inside batch
optimizer
###### 254
inside loop
[437 678  70 221 201 397 208 178 531 264]
inside batch
optimizer
###### 255
inside loop
[678  70 221 201 397 208 178 531 264 760]
inside batch
optimizer
###### 256
inside loop

optimizer
###### 339
inside loop
[796 694 551 756 693 644 440 139 757 562]
inside batch
optimizer
###### 340
inside loop
[694 551 756 693 644 440 139 757 562 107]
inside batch
optimizer
###### 341
inside loop
[551 756 693 644 440 139 757 562 107 351]
inside batch
optimizer
###### 342
inside loop
[756 693 644 440 139 757 562 107 351 597]
inside batch
optimizer
###### 343
inside loop
[693 644 440 139 757 562 107 351 597 159]
inside batch
optimizer
###### 344
inside loop
[644 440 139 757 562 107 351 597 159 132]
inside batch
optimizer
###### 345
inside loop
[440 139 757 562 107 351 597 159 132 592]
inside batch
optimizer
###### 346
inside loop
[139 757 562 107 351 597 159 132 592 174]
inside batch
optimizer
###### 347
inside loop
[757 562 107 351 597 159 132 592 174 657]
inside batch
optimizer
###### 348
inside loop
[562 107 351 597 159 132 592 174 657 667]
inside batch
optimizer
###### 349
inside loop
[107 351 597 159 132 592 174 657 667   4]
inside batch
optimizer
###### 350
inside loop

optimizer
###### 433
inside loop
[336 244 538 575 550 405 732 741 184 226]
inside batch
optimizer
###### 434
inside loop
[244 538 575 550 405 732 741 184 226 585]
inside batch
optimizer
###### 435
inside loop
[538 575 550 405 732 741 184 226 585 327]
inside batch
optimizer
###### 436
inside loop
[575 550 405 732 741 184 226 585 327 573]
inside batch
optimizer
###### 437
inside loop
[550 405 732 741 184 226 585 327 573 108]
inside batch
optimizer
###### 438
inside loop
[405 732 741 184 226 585 327 573 108 672]
inside batch
optimizer
###### 439
inside loop
[732 741 184 226 585 327 573 108 672 172]
inside batch
optimizer
###### 440
inside loop
[741 184 226 585 327 573 108 672 172 579]
inside batch
optimizer
###### 441
inside loop
[184 226 585 327 573 108 672 172 579 363]
inside batch
optimizer
###### 442
inside loop
[226 585 327 573 108 672 172 579 363 735]
inside batch
optimizer
###### 443
inside loop
[585 327 573 108 672 172 579 363 735  44]
inside batch
optimizer
###### 444
inside loop

optimizer
###### 527
inside loop
[474 200  55 401 239 719 230 519 461 234]
inside batch
optimizer
###### 528
inside loop
[200  55 401 239 719 230 519 461 234 176]
inside batch
optimizer
###### 529
inside loop
[ 55 401 239 719 230 519 461 234 176 673]
inside batch
optimizer
###### 530
inside loop
[401 239 719 230 519 461 234 176 673 294]
inside batch
optimizer
###### 531
inside loop
[239 719 230 519 461 234 176 673 294 724]
inside batch
optimizer
###### 532
inside loop
[719 230 519 461 234 176 673 294 724 231]
inside batch
optimizer
###### 533
inside loop
[230 519 461 234 176 673 294 724 231 367]
inside batch
optimizer
###### 534
inside loop
[519 461 234 176 673 294 724 231 367 727]
inside batch
optimizer
###### 535
inside loop
[461 234 176 673 294 724 231 367 727 540]
inside batch
optimizer
###### 536
inside loop
[234 176 673 294 724 231 367 727 540  67]
inside batch
optimizer
###### 537
inside loop
[176 673 294 724 231 367 727 540  67 422]
inside batch
optimizer
###### 538
inside loop

optimizer
###### 621
inside loop
[616 728 344 594 541 257 194 788 755 495]
inside batch
optimizer
###### 622
inside loop
[728 344 594 541 257 194 788 755 495 475]
inside batch
optimizer
###### 623
inside loop
[344 594 541 257 194 788 755 495 475 780]
inside batch
optimizer
###### 624
inside loop
[594 541 257 194 788 755 495 475 780 709]
inside batch
optimizer
###### 625
inside loop
[541 257 194 788 755 495 475 780 709 136]
inside batch
optimizer
###### 626
inside loop
[257 194 788 755 495 475 780 709 136 369]
inside batch
optimizer
###### 627
inside loop
[194 788 755 495 475 780 709 136 369 643]
inside batch
optimizer
###### 628
inside loop
[788 755 495 475 780 709 136 369 643 542]
inside batch
optimizer
###### 629
inside loop
[755 495 475 780 709 136 369 643 542 509]
inside batch
optimizer
###### 630
inside loop
[495 475 780 709 136 369 643 542 509 303]
inside batch
optimizer
###### 631
inside loop
[475 780 709 136 369 643 542 509 303 748]
inside batch
optimizer
###### 632
inside loop

KeyboardInterrupt: 

In [157]:
x_train[[386,565,570,75,445,234,375,103,216,196]].shape

(10, 28, 28, 1)

In [ ]:
[]